<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, learning_curve
from xgboost import XGBClassifier

In [0]:
import matplotlib.pyplot as plt
from IPython.display import FileLink
import seaborn as sns
import pickle

In [22]:
!pwd
!ls -l
!ls -lR ../input/

/content/working
total 116
-rw-r--r-- 1 root root 116440 Oct  6 11:49 grid.pkl
../input/:
total 4
drwxr-xr-x 2 root root 4096 Oct  6 10:12 mlcourse-ai-flight-delays-xgboost-data

../input/mlcourse-ai-flight-delays-xgboost-data:
total 241448
-rw-r--r-- 1 root root 245630891 Oct  6 10:12 train_enc.h5
-rw-r--r-- 1 root root   1605136 Oct  6 10:12 y.h5


**Read previously saved data**

In [0]:
#train_df = pd.read_hdf('../input/mlcourse-ai-flight-delays-xgboost-data/train.h5')
#test_df = pd.read_hdf('../input/mlcourse-ai-flight-delays-xgboost-data/test.h5')

In [0]:
train_df = pd.read_hdf('../input/mlcourse-ai-flight-delays-xgboost-data/train_enc.h5')
#test_df = pd.read_hdf('../input/mlcourse-ai-flight-delays-xgboost-data/test_enc.h5')
y = pd.read_hdf('../input/mlcourse-ai-flight-delays-xgboost-data/y.h5')

**Train XGBoost**

In [25]:
%%time
# measure performance (GPU)
XGBClassifier(tree_method='gpu_hist').fit(train_df[:10000], y[:10000])

CPU times: user 1.18 s, sys: 825 ms, total: 2 s
Wall time: 2 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [26]:
%%time
# measure performance (CPU)
XGBClassifier().fit(train_df[:10000], y[:10000])

CPU times: user 7.04 s, sys: 15.9 ms, total: 7.06 s
Wall time: 7.07 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Use different **random_state** here.

In [27]:
%%time
params = {'max_depth': range(1, 11), 'n_estimators': range(1, 11)}
grid = GridSearchCV(XGBClassifier(random_state=17, tree_method='gpu_hist'), params, cv=5,
                    scoring='roc_auc', verbose=3, return_train_score=True)
grid.fit(train_df, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] max_depth=1, n_estimators=1 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=1, n_estimators=1, score=(train=0.808, test=0.812), total=   1.1s
[CV] max_depth=1, n_estimators=1 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  max_depth=1, n_estimators=1, score=(train=0.808, test=0.809), total=   1.1s
[CV] max_depth=1, n_estimators=1 .....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


[CV]  max_depth=1, n_estimators=1, score=(train=0.808, test=0.811), total=   1.1s
[CV] max_depth=1, n_estimators=1 .....................................
[CV]  max_depth=1, n_estimators=1, score=(train=0.809, test=0.805), total=   1.1s
[CV] max_depth=1, n_estimators=1 .....................................
[CV]  max_depth=1, n_estimators=1, score=(train=0.810, test=0.805), total=   1.0s
[CV] max_depth=1, n_estimators=2 .....................................
[CV]  max_depth=1, n_estimators=2, score=(train=0.848, test=0.853), total=   1.1s
[CV] max_depth=1, n_estimators=2 .....................................
[CV]  max_depth=1, n_estimators=2, score=(train=0.849, test=0.849), total=   1.1s
[CV] max_depth=1, n_estimators=2 .....................................
[CV]  max_depth=1, n_estimators=2, score=(train=0.849, test=0.851), total=   1.1s
[CV] max_depth=1, n_estimators=2 .....................................
[CV]  max_depth=1, n_estimators=2, score=(train=0.850, test=0.846), total=   1.2s


KeyboardInterrupt: ignored

In [0]:
pickle.dump(grid, open("grid.pkl", "wb"))
FileLink('grid.pkl')

In [0]:
plt.figure(figsize=(40,4))
xx = [str(x) for x in grid.cv_results_['params']]
plt.plot(xx, grid.cv_results_['mean_test_score'], c='r')
plt.fill_between(xx, grid.cv_results_['mean_test_score'] - grid.cv_results_['std_test_score'],
                 grid.cv_results_['mean_test_score'] + grid.cv_results_['std_test_score'],
                 alpha=0.1, color="r")
plt.plot(xx, grid.cv_results_['mean_train_score'], c='g')
plt.fill_between(xx, grid.cv_results_['mean_train_score'] - grid.cv_results_['std_train_score'],
                 grid.cv_results_['mean_train_score'] + grid.cv_results_['std_train_score'],
                 alpha=0.1, color="g")
plt.xticks(rotation=90)
plt.title('Validation curves')
plt.axhline(0.8)
plt.axhline(0.9)
plt.axhline(0.95)
plt.show()

In [0]:
plt.figure(figsize=(16,4))
xx, train_scores, test_scores = learning_curve(grid.best_estimator_, train_df, y,
  train_sizes=np.linspace(0.01, 0.99, 50), cv=5, scoring='roc_auc')
plt.plot(xx, np.mean(train_scores, axis=1), c='g')
plt.fill_between(xx, np.mean(train_scores, axis=1) - np.std(train_scores, axis=1),
                 np.mean(train_scores, axis=1) + np.std(train_scores, axis=1),
                 alpha=0.1, color="g")
plt.plot(xx, np.mean(test_scores, axis=1), c='r')
plt.fill_between(xx, np.mean(test_scores, axis=1) - np.std(test_scores, axis=1),
                 np.mean(test_scores, axis=1) + np.std(test_scores, axis=1),
                 alpha=0.1, color="r")
plt.title('Learning curves')
plt.show()

In [0]:
plt.figure(figsize=(80,4))
plt.bar(train_df.columns, grid.best_estimator_.feature_importances_)
plt.xticks(rotation=90)
plt.title('Feature importance')
plt.show()

In [0]:
grid.best_params_

In [0]:
grid.best_score_

Now's your turn! Go and improve the model to beat **"A2 baseline (10 credits)"** - **0.75914** LB score. It's crucial to come up with some good features. 

For discussions, stick to the **#a2_kaggle_fall2019** thread in the **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com) channel. Serhii Romanenko (@serhii_romanenko) will be there to help. 

Welcome to Kaggle!

![img](https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg)
*from the ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963) thread.*

In [1]:
# prepare Kaggle environment
!pwd
!mkdir -pv working
!mkdir -pv input/mlcourse-ai-fall-2019-xgboost-data

import os
os.chdir('working')
!pwd

!wget -P ../input/mlcourse-ai-flight-delays-xgboost-data/ 'https://www.kaggleusercontent.com/kf/21546212/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..oGUENgY6FetSf3xcVzwvIQ.2G5nm23SjtlwazyDurU3U3eTLFxoSZrvLFUi0amJ6-hqqAYrZ0tpamKUuBQu9YxLYlf159-qltrdCiCqPbwp0_1Xk-dvpTXnKZN1V3nnQOGjpvW9YTfPtOt0Pbk4rhRyB3XDUx_AF38n9htbPgC3upzRyc20Ws2JyFpc3mRsmtnzyw7KitLXJ6QJFzU1RSDd.Y-PMozbYpYl3GSu9NYMhfg/train_enc.h5'
!wget -P ../input/mlcourse-ai-flight-delays-xgboost-data/ 'https://www.kaggleusercontent.com/kf/21546212/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..WocYsnt7wg0U7Ee5DcHyHQ.bkfjtPQsF6xpfV9mA0DehJ645BJgzXnPyi8re8mWVrG_QD9cnHhNqkcK0cCnvA7YFRlWNh09_OkKEfGtfNIfQk-hPEO3LDJPQfpkdEFQ_v6Qlt4UVvt7UQ6pesTWcQNFb3qpMCZSeMhqcvkxHUYOxlCuStqQ3CHCtRoiEBDFljxyY6A2oBXtMJJHth6AHqkq.ky_7fO310ZAD2vWRDfI-dA/y.h5'

!pip uninstall -y tables
!pip install tables

/content
mkdir: created directory 'working'
mkdir: created directory 'input'
mkdir: created directory 'input/mlcourse-ai-fall-2019-xgboost-data'
/content/working
--2019-10-06 10:11:51--  https://www.kaggleusercontent.com/kf/21546212/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..oGUENgY6FetSf3xcVzwvIQ.2G5nm23SjtlwazyDurU3U3eTLFxoSZrvLFUi0amJ6-hqqAYrZ0tpamKUuBQu9YxLYlf159-qltrdCiCqPbwp0_1Xk-dvpTXnKZN1V3nnQOGjpvW9YTfPtOt0Pbk4rhRyB3XDUx_AF38n9htbPgC3upzRyc20Ws2JyFpc3mRsmtnzyw7KitLXJ6QJFzU1RSDd.Y-PMozbYpYl3GSu9NYMhfg/train_enc.h5
Resolving www.kaggleusercontent.com (www.kaggleusercontent.com)... 35.190.26.106
Connecting to www.kaggleusercontent.com (www.kaggleusercontent.com)|35.190.26.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245630891 (234M) [application/octet-stream]
Saving to: ‘../input/mlcourse-ai-fall-2019-xgboost-data/train_enc.h5’

train_enc.h5        100%[===================>] 234.25M  39.5MB/s    in 6.6s    

2019-10-06 10:12:03 (35.6 MB/s) -

In [1]:
# prepare Kaggle envirnment after restart
!pwd
import os
os.chdir('working')
!pwd

/content
/content/working
